In [1]:
import sys
sys.path.insert(1, '../')  # to load from any submodule in the repo

from utils import reader
from utils import dpcrUtils as utils
from utils import generator
from utils import reconstructor
from models import models

from training import train_detector
from training import train_predictor

import torch
import torch.nn

import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
import time
from mpl_toolkits.mplot3d import Axes3D
import itertools
import _pickle as cPickle
from torchviz import make_dot

device = utils.getDevice()
print ("Device:", device)
    
torch.no_grad()

seed_file = open('../utils/seed.txt', "r")
seed = int(seed_file.read())
seed_file.close()

print ("Using Seed:", seed)
    
torch.manual_seed(seed)
np.random.seed(seed)

Using 1 CUDA devices (GeForce GTX 1070)
Device: cuda
Using Seed: 34897567


In [2]:
validation_set = {
    'multi_simple_shapes':{
        'predictor':models.loadModel('../hpc/results/predictor_multi_simple_shapes_cnet_mish_radam_sd/2020-12-19_062318/predictor_checkpoints.t7', device=torch.device('cpu')),
        'detector':models.loadModel('../hpc/results/detector_multi_simple_shapes_cnet_mish_radam_sd/2020-12-15_224339/detector_checkpoints.t7', device=torch.device('cpu'))
    },
    'multi_faces' :{
        'predictor':models.loadModel('../hpc/results/predictor_multi_faces_cnet_mish_radam_sd/2020-12-15_224134/predictor_checkpoints.t7', device=torch.device('cpu')),
        'detector':models.loadModel('../hpc/results/detector_multi_faces_cnet_mish_radam_sd/2020-12-09_074025/detector_checkpoints.t7', device=torch.device('cpu'))
    },
    'single_armadillo' :{
        'predictor':models.loadModel('../hpc/results/predictor_single_armadillo_cnet_mish_radam_sd/2020-12-15_224123/predictor_checkpoints.t7', device=torch.device('cpu')),
        'detector':models.loadModel('../hpc/results/detector_single_armadillo_cnet_mish_radam_sd/2020-12-15_224123/detector_checkpoints.t7', device=torch.device('cpu'))
    },
    'single_bunny' :{
        'predictor':models.loadModel('../hpc/results/predictor_single_bunny_cnet_mish_radam/2020-12-31_171657/predictor_checkpoints.t7', device=torch.device('cpu')),
        'detector':models.loadModel('../hpc/results/detector_single_bunny_cnet_mish_radam_sd/2020-12-08_110946/detector_checkpoints.t7', device=torch.device('cpu'))
    },
    'multi_cuboids' :{
        'predictor':models.loadModel('../hpc/results/predictor_multi_cuboids_cnet_mish_radam_sd/2020-12-19_062203/predictor_checkpoints.t7', device=torch.device('cpu')),
        'detector':models.loadModel('../hpc/results/detector_multi_cuboids_cnet_mish_radam_sd/2020-12-15_224131/detector_checkpoints.t7', device=torch.device('cpu'))
    },
    'multi_ellipsoids' :{
        'predictor':models.loadModel('../hpc/results/predictor_multi_ellipsoids_cnet_mish_radam_sd/2020-12-19_062200/predictor_checkpoints.t7', device=torch.device('cpu')),
        'detector':models.loadModel('../hpc/results/detector_multi_ellipsoids_cnet_mish_radam_sd/2020-12-15_224150/detector_checkpoints.t7', device=torch.device('cpu'))
    },
    'multi_polyhedrons' :{
        'predictor':models.loadModel('../hpc/results/predictor_multi_cuboids_cnet_mish_radam_sd/2020-12-19_062203/predictor_checkpoints.t7', device=torch.device('cpu')),
        'detector':models.loadModel('../hpc/results/detector_multi_polyhedrons_cnet_mish_radam_sd/2020-12-15_224129/detector_checkpoints.t7', device=torch.device('cpu'))
    }

}

torch.cuda.empty_cache()

print ("Load complete!")

Load complete!


In [3]:
for dataset in validation_set:

    val_model_paths = ['../' + path for path in generator.getPaths()[dataset]['val']]
    
    hidden_range = (6,11)
    iteration_range = (6,9)
    
    print ("\nDataset: ", dataset, "---------------------\n")

    print ("\nReading validation models.. \n")
    val_models, val_knns = generator.readModels(val_model_paths, device=device, incrementKNNid = False)
    
    validation_size = int(30 / len(val_models))

    print ("\nGenerating validation samples for %d models.." % (len(val_models)))
    start = time.time()
    val_samples, val_sample_stats = generator.getSamples(val_knns, validation_size, hidden_range = hidden_range, iteration_range = iteration_range)
    print ("   ..done! (%.1fs) ---> #samples: %d" % (time.time() - start, len(val_samples)))
    
    val_models_tensor = torch.cat(val_models)

    # increment indices to concat
    val_knn_sizes = [knn.size(0) for knn in val_knns]
    val_knns = [knn + sum(val_knn_sizes[:i]) for (i, knn) in enumerate(val_knns)]

    val_knns_tensor = torch.cat(val_knns).cpu()
    
    validation_set[dataset]['pts'] = val_models_tensor
    validation_set[dataset]['knn'] = val_knns_tensor
    validation_set[dataset]['samples'] = val_samples


Dataset:  multi_simple_shapes ---------------------


Reading validation models.. 

Processing model 1 of 9..
   ..done! (1.3s)
Processing model 2 of 9..
   ..done! (0.1s)
Processing model 3 of 9..
   ..done! (0.1s)
Processing model 4 of 9..
   ..done! (0.1s)
Processing model 5 of 9..
   ..done! (0.1s)
Processing model 6 of 9..
   ..done! (0.1s)
Processing model 7 of 9..
   ..done! (0.1s)
Processing model 8 of 9..
   ..done! (0.1s)
Processing model 9 of 9..
   ..done! (0.1s)

Generating validation samples for 9 models..
   ..done! (0.2s) ---> #samples: 207

Dataset:  multi_faces ---------------------


Reading validation models.. 

Processing model 1 of 2..
   ..done! (0.3s)
Processing model 2 of 2..
   ..done! (0.3s)

Generating validation samples for 2 models..
   ..done! (0.3s) ---> #samples: 244

Dataset:  single_armadillo ---------------------


Reading validation models.. 

Processing model 1 of 1..
   ..done! (0.4s)

Generating validation samples for 1 models..
   ..done! (0.3s

In [5]:
with torch.no_grad():

    for dataset in validation_set:

        torch.cuda.empty_cache()

        print ("Measuring: ", dataset)

        predictor = validation_set[dataset]['predictor'].to(device)
        detector = validation_set[dataset]['detector'].to(device)

        predictor.eval()
        detector.eval()

        val_pts = validation_set[dataset]['pts'].to(device)
        val_samples = validation_set[dataset]['samples']
        
        total_prediction_time = 0
        total_detection_time = 0
        total_corrector_time = 0
        total_time = 0
        
        for sample in val_samples:
            
            start = time.time()

            prediction_time, detection_time, corrector_time = reconstructor.reconstruct(
                val_pts[sample[:,0]],
                predictor,
                detector,
                corrector = None,
                max_iters = 10,
                t = 0,
                verbose=False,
                device = device,
                corrector_stage=5,
                returnTimes=True
            )
            
            end = time.time()
            
            total_time += end-start
            total_prediction_time += prediction_time
            total_detection_time += detection_time
            total_corrector_time += corrector_time
            
        print ("AVG total time: %.5fs" % (total_time / len(val_samples)))
        print ("AVG prediction time: %.5fs" % (total_prediction_time / len(val_samples)))
        print ("AVG detection time: %.5fs" % (total_detection_time / len(val_samples)))
        print ("AVG corrector time: %.5fs" % (total_corrector_time / len(val_samples)))
    
    

Measuring:  multi_simple_shapes
AVG total time: 0.48798s
AVG prediction time: 0.10731s
AVG detection time: 0.01258s
AVG corrector time: 0.00000s
Measuring:  multi_faces
AVG total time: 3.98132s
AVG prediction time: 0.18099s
AVG detection time: 0.02148s
AVG corrector time: 0.00000s
Measuring:  single_armadillo


KeyboardInterrupt: 

validation_results = {}

perms_np = np.array(list(itertools.permutations(np.arange(6))))
perms = torch.from_numpy(perms_np).long().to(device)

with torch.no_grad():

    for dataset in validation_set:
        
        torch.cuda.empty_cache()

        validation_results[dataset] = {}

        print ("Validating: ", dataset)

        predictor = validation_set[dataset]['predictor'].to(device)
        detector = validation_set[dataset]['detector'].to(device)
        
        predictor.eval()
        detector.eval()

        for val_dataset in validation_set:
            
            torch.cuda.empty_cache()

            validation_results[dataset][val_dataset] = {}

            val_pts = validation_set[val_dataset]['pts'].to(device)
            val_knn = validation_set[val_dataset]['knn']
            val_samples = validation_set[val_dataset]['samples']

            val_neighbors_dirs = (val_pts[val_knn] - val_pts[:, None, :]).to(device)

            av_c1_freq = sum([torch.sum(sample[:,1]).item() for sample in val_samples]) / sum([sample.size(0) for sample in val_samples])
            class_weights = torch.tensor([av_c1_freq, 1-av_c1_freq]).float().to(device)

            print ("   Validating for: ", val_dataset)

            start = time.time()
            det_loss, det_acc, det_acc_c0, det_acc_c1 = train_detector.getTestLoss(val_pts, val_samples, detector, class_weights)

            print ("      -> detector done! (%.1f)" % (time.time() - start))
            print ("         Detector Loss:", det_loss)
            print ("         Detector Acc:", det_acc)
            print ("         Detector C0 Acc:", det_acc_c0)
            print ("         Detector C1 Acc:", det_acc_c1)

            start = time.time()
            pred_loss = train_predictor.val_loss = train_predictor.getTestLoss(val_pts, val_samples, predictor, val_neighbors_dirs, perms, torch.nn.MSELoss(reduction='mean'))

            print ("      -> predictor done! (%.1f)" % (time.time() - start))
            print ("         Predictor Loss:", pred_loss)

            print ("")

            validation_results[dataset][val_dataset]['det_loss'] = det_loss
            validation_results[dataset][val_dataset]['det_acc'] = det_acc
            validation_results[dataset][val_dataset]['det_acc_c0'] = det_acc_c0
            validation_results[dataset][val_dataset]['det_acc_c1'] = det_acc_c1
            validation_results[dataset][val_dataset]['pred_loss'] = pred_loss


In [32]:
validation_results

{'multi_simple_shapes': {}}